<a href="https://colab.research.google.com/github/karanbhatt1/ConversationalChatBot/blob/main/ArtificalIntelligenceProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OBECTIVE:
> To build a conversational AI which takes the input data like pdf, txt,image,json and answer the questions related to it.


# Installing the useful libraries

In [ ]:
!pip install langchain langchain_community langchain_chroma

In [ ]:
!pip install chromadb

In [ ]:
!pip install pypdf

# Importing the dependencies

In [80]:
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_text_splitters import CharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers.string import StrOutputParser
from pypdf import PdfReader

# Step 2 -- Document Processing

## 1. Taking a plain text file from user
> Merging the text in the document.

1-> **!pip install pypdf [to handle pdf files]**
</br>
2-> **from pypdf import PdfReader**
</br>
3-> **readerObject.pages** to get toatal pages in pdf
<br>
4-> **.extract_text()**: to extract text from particular page

In [160]:
fldr = "/content/drive/MyDrive/machinelearningprojects/AIProject/Atomic habits ( PDFDrive ).pdf"

In [161]:
pdfFile = pypdf.PdfReader(fldr)

In [162]:
len(pdfFile.pages)

256

In [170]:
# concatinating the text:
combinedText = ""
for page in pdfdir.pages:
  combinedText = combinedText + page.extract_text() + "\n"

In [171]:
combinedText

"\n\nAN\tIMPRINT\tOF\tPENGUIN\tRANDOM\tHOUSE\tLLC\n375\tHudson\tStreet\nNew\tYork,\tNew\tYork\t10014\nCopyright\t©\t2018\tby\tJames\tClear\nPenguin\tsupports\tcopyright.\tCopyright\tfuels\tcreativity,\tencourages\tdiverse\tvoices,\tpromotes\tfree\tspeech,\tand\tcreates\ta\tvibrant\tculture.\tThank\tyou\tfor\tbuying\tan\tauthorized\tedition\tof\tthis\tbook\tand\tfor\ncomplying\twith\tcopyright\tlaws\tby\tnot\treproducing,\tscanning,\tor\tdistributing\tany\tpart\tof\tit\tin\tany\tform\twithout\tpermission.\tYou\tare\tsupporting\twriters\tand\tallowing\tPenguin\tto\tcontinue\tto\tpublish\tbooks\nfor\tevery\treader.\nEbook\tISBN\t9780735211308\nWhile\tthe\tauthor\thas\tmade\tevery\teffort\tto\tprovide\taccurate\tInternet\taddresses\tat\tthe\ttime\tof\tpublication,\tneither\tthe\tpublisher\tnor\tthe\tauthor\tassumes\tany\tresponsibility\tfor\terrors,\tor\tfor\tchanges\tthat\toccur\nafter\tpublication.\tFurther,\tthe\tpublisher\tdoes\tnot\thave\tany\tcontrol\tover\tand\tdoes\tnot\tassume\tan

In [172]:
len(combinedText)

464032

# Step 3 Splitting the text

> Total chunks = 28 with size of each chunk as 5000

In [173]:
len(combinedText) # len = four lakhs sixty three thousand seven hundred seventy six token or text

464032

In [174]:
def chunk_size(combinedText):
  if(len(combinedText)>250000):
    return 70000
  elif(len(combinedText)>100000):
    return 10000
  elif(len(combinedText)>50000):
    return 5000
  elif(len(combinedText)>25000):
    return 1000
  else:
    return 500

In [181]:
splitter = CharacterTextSplitter(chunk_size= 5000,
                                 chunk_overlap = 500,
                                   length_function = len
                                   )

In [182]:
splited_chunks = splitter.split_text(combinedText)

In [183]:
splited_chunks

['AN\tIMPRINT\tOF\tPENGUIN\tRANDOM\tHOUSE\tLLC\n375\tHudson\tStreet\nNew\tYork,\tNew\tYork\t10014\nCopyright\t©\t2018\tby\tJames\tClear\nPenguin\tsupports\tcopyright.\tCopyright\tfuels\tcreativity,\tencourages\tdiverse\tvoices,\tpromotes\tfree\tspeech,\tand\tcreates\ta\tvibrant\tculture.\tThank\tyou\tfor\tbuying\tan\tauthorized\tedition\tof\tthis\tbook\tand\tfor\ncomplying\twith\tcopyright\tlaws\tby\tnot\treproducing,\tscanning,\tor\tdistributing\tany\tpart\tof\tit\tin\tany\tform\twithout\tpermission.\tYou\tare\tsupporting\twriters\tand\tallowing\tPenguin\tto\tcontinue\tto\tpublish\tbooks\nfor\tevery\treader.\nEbook\tISBN\t9780735211308\nWhile\tthe\tauthor\thas\tmade\tevery\teffort\tto\tprovide\taccurate\tInternet\taddresses\tat\tthe\ttime\tof\tpublication,\tneither\tthe\tpublisher\tnor\tthe\tauthor\tassumes\tany\tresponsibility\tfor\terrors,\tor\tfor\tchanges\tthat\toccur\nafter\tpublication.\tFurther,\tthe\tpublisher\tdoes\tnot\thave\tany\tcontrol\tover\tand\tdoes\tnot\tassume\tany\t

In [184]:
len(splited_chunks)

28

In [185]:
stxt = splitter.create_documents([combinedText])

In [187]:
len(stxt) # 28 chunks

28

# Step 4 Making Database and adding chunks in collection

In [121]:
import chromadb

In [122]:
clt = chromadb.Client()

In [123]:
vdb = clt.create_collection(name= "chkdatacoll")

In [ ]:
vdb.add([])

# Step 4 : Import the data using Embedding models
> BERT, GPT , HuggingFace Embedding, OpenAI EMbedings

In [194]:
from langchain.embeddings  import HuggingFaceEmbeddings